# First Class Patterns

In [1]:
#require "core"
open Core

/Users/theowang/.opam/4.12.0/lib/base/base_internalhash_types: added to search path
/Users/theowang/.opam/4.12.0/lib/base/base_internalhash_types/base_internalhash_types.cma: loaded
/Users/theowang/.opam/4.12.0/lib/base/caml: added to search path
/Users/theowang/.opam/4.12.0/lib/base/caml/caml.cma: loaded
/Users/theowang/.opam/4.12.0/lib/base/shadow_stdlib: added to search path
/Users/theowang/.opam/4.12.0/lib/base/shadow_stdlib/shadow_stdlib.cma: loaded
/Users/theowang/.opam/4.12.0/lib/sexplib0: added to search path
/Users/theowang/.opam/4.12.0/lib/sexplib0/sexplib0.cma: loaded
/Users/theowang/.opam/4.12.0/lib/base: added to search path
/Users/theowang/.opam/4.12.0/lib/base/base.cma: loaded
/Users/theowang/.opam/4.12.0/lib/base/md5: added to search path
/Users/theowang/.opam/4.12.0/lib/base/md5/md5_lib.cma: loaded
/Users/theowang/.opam/4.12.0/lib/fieldslib: added to search path
/Users/theowang/.opam/4.12.0/lib/fieldslib/fieldslib.cma: loaded
/Users/theowang/.opam/4.12.0/lib/ppx_compar

In [2]:
type (_, _, _) pat = 
    Any : ('a, 'r, 'r) pat
  | Int : int -> (int, 'r, 'r) pat
  | Var : ('a, 'a -> 'r, 'r) pat (* Would be 'a code here instead *)
  | EmptyList : ('a list, 'r, 'r) pat
  | Pair : ('a, 'k, 'j) pat * ('b, 'j, 'r) pat -> ('a * 'b, 'k, 'r) pat
  | Cons : ('a, 'k, 'j) pat * ('a list, 'j, 'r) pat -> ('a list, 'k, 'r) pat

type (_, _, _) pat =
    Any : ('a, 'r, 'r) pat
  | Int : int -> (int, 'r, 'r) pat
  | Var : ('a, 'a -> 'r, 'r) pat
  | EmptyList : ('a list, 'r, 'r) pat
  | Pair : ('a, 'k, 'j) pat * ('b, 'j, 'r) pat -> ('a * 'b, 'k, 'r) pat
  | Cons : ('a, 'k, 'j) pat * ('a list, 'j, 'r) pat -> ('a list, 'k, 'r) pat


In [3]:
let rec match_ : type a f r. a -> (a, f, r) pat -> f -> r = (* Explicit type notation *)
  fun scrutinee pattern cont -> 
    match pattern, scrutinee with
    | Any, _ -> cont
    | Int i, _ -> if i = scrutinee then cont else failwith "match failed"
    | Var, _ -> cont scrutinee
    | EmptyList, _ -> cont
    | Pair (p1, p2), (a1, a2) -> 
      let m1 = match_ a1 p1 cont in
      let m2 = match_ a2 p2 m1 in
      m2
    | Cons (p1, p2), a1::a2 -> 
        let m1 = match_ a1 p1 cont in
        let m2 = match_ a2 p2 m1 in
        m2
    | _, _ -> failwith "match failed"

val match_ : 'a -> ('a, 'f, 'r) pat -> 'f -> 'r = <fun>


# Tagless final embedding

In [4]:
module type Symantics = sig
  type ('c, 'dv) repr
  val int : int  -> ('c, int) repr
  val bool: bool -> ('c, bool) repr
  val lam : (('c, 'da) repr -> ('c, 'db) repr) -> ('c, 'da -> 'db) repr
  val app : ('c, 'da -> 'db) repr -> ('c, 'da) repr -> ('c, 'db) repr
  val fix : ('x -> 'x) -> (('c, 'da -> 'db) repr as 'x)
  val add : ('c, int) repr -> ('c, int) repr -> ('c, int) repr
  val mul : ('c, int) repr -> ('c, int) repr -> ('c, int) repr
  val leq : ('c, int) repr -> ('c, int) repr -> ('c, bool) repr
  val if_ : ('c, bool) repr -> (unit -> 'x) -> (unit -> 'x) -> (('c, 'da) repr as 'x)
end

module type Symantics =
  sig
    type ('c, 'dv) repr
    val int : int -> ('c, int) repr
    val bool : bool -> ('c, bool) repr
    val lam : (('c, 'da) repr -> ('c, 'db) repr) -> ('c, 'da -> 'db) repr
    val app : ('c, 'da -> 'db) repr -> ('c, 'da) repr -> ('c, 'db) repr
    val fix :
      (('c, 'da -> 'db) repr -> ('c, 'da -> 'db) repr) ->
      ('c, 'da -> 'db) repr
    val add : ('c, int) repr -> ('c, int) repr -> ('c, int) repr
    val mul : ('c, int) repr -> ('c, int) repr -> ('c, int) repr
    val leq : ('c, int) repr -> ('c, int) repr -> ('c, bool) repr
    val if_ :
      ('c, bool) repr ->
      (unit -> ('c, 'da) repr) -> (unit -> ('c, 'da) repr) -> ('c, 'da) repr
  end


In [5]:
(* https://okmij.org/ftp/tagless-final/JFP.pdf *)
module Interpreter : Symantics = struct
  type ('c, 'dv) repr = 'dv
  let int x = x
  let bool b = b
  let lam f = f
  let app e1 e2 = e1 e2 
  let fix f = let rec self n = f self n in self
  let add e1 e2 = e1 + e2
  let mul e1 e2 = e1 * e2
  let leq e1 e2 = e1 <= e2
  let if_ eb et ee = if eb then et () else ee ()
end

module Interpreter : Symantics


In [6]:
let module I = Interpreter in I.app (I.lam (fun x -> x)) (I.bool true)

- : ('_weak1, bool) Interpreter.repr = <abstr>


In [7]:
module type Pat = sig
  type ('c, 'a, 'f, 'r) pat
  val int_ : int -> ('c, int, 'r, 'r) pat
  val any_ : ('c, 'a, 'r, 'r) pat
  val var : ('c, 'a, 'a -> 'r, 'r) pat
  val emptylist : ('c, 'a list, 'r, 'r) pat
  val pair : ('c, 'a, 'k, 'j) pat -> ('c, 'b, 'j, 'r) pat -> ('c, 'a * 'b, 'k, 'r) pat
  val cons : ('c, 'a, 'k, 'j) pat -> ('c, 'a list, 'j, 'r) pat -> ('c, 'a list, 'k, 'r) pat
  val or_ : ('c, 'a, 'k, 'r) pat -> ('c, 'a, 'k, 'r) pat -> ('c, 'a, 'k, 'r) pat
end

module type Pat =
  sig
    type ('c, 'a, 'f, 'r) pat
    val int_ : int -> ('c, int, 'r, 'r) pat
    val any_ : ('c, 'a, 'r, 'r) pat
    val var : ('c, 'a, 'a -> 'r, 'r) pat
    val emptylist : ('c, 'a list, 'r, 'r) pat
    val pair :
      ('c, 'a, 'k, 'j) pat ->
      ('c, 'b, 'j, 'r) pat -> ('c, 'a * 'b, 'k, 'r) pat
    val cons :
      ('c, 'a, 'k, 'j) pat ->
      ('c, 'a list, 'j, 'r) pat -> ('c, 'a list, 'k, 'r) pat
    val or_ :
      ('c, 'a, 'k, 'r) pat -> ('c, 'a, 'k, 'r) pat -> ('c, 'a, 'k, 'r) pat
  end


In [33]:
module PattnMatch : Pat = struct
  open Core.Option.Monad_infix
  type ('c, 'a, 'f, 'r) pat = 'a -> 'f -> 'r option
  let int_ i scrutinee cont = if i = scrutinee then Some (cont) else None
  let any_ scrutinee cont = Some cont
  let var scrutinee cont = Some (cont scrutinee)
  let emptylist scrutinee cont = Some cont
  let pair constr_left constr_right (scrutinee_left, scrutinee_right) cont = 
    constr_left scrutinee_left cont >>= fun m1 -> 
    constr_right scrutinee_right m1 >>= fun m2 -> 
    Some m2
  let cons constr_left constr_right (scrutinee) cont = 
    match scrutinee with 
    | scrutinee_left :: scrutinee_right ->
      constr_left scrutinee_left cont >>= fun m1 -> 
      constr_right scrutinee_right m1 >>= fun m2 ->
      Some m2
    | _ -> None
  let or_ constr_left constr_right scrutinee cont = 
      match constr_left scrutinee cont with
      | Some res -> Some res
      | None -> constr_right scrutinee cont
  let rename constr scrutinee cont = constr scrutinee (fun x y -> cont y x)
  (* TODO: implement this sort of renaming *)
end

module PattnMatch : Pat


In [9]:
PattnMatch.pair PattnMatch.var PattnMatch.var (1, 2) (fun x -> fun y -> x + y)

- : int Base.Option.t = Base.Option.Some 3


In [10]:
PattnMatch.or_ (PattnMatch.pair PattnMatch.var (PattnMatch.int_ (3))) (PattnMatch.pair (PattnMatch.int_ (3)) PattnMatch.var) (3, 2) (fun x -> x)

- : Core.Int.t Base.Option.t = Base.Option.Some 2


In [11]:
PattnMatch.pair PattnMatch.var PattnMatch.var

- : '_weak2 * '_weak3 ->
    ('_weak2 -> '_weak3 -> '_weak4) -> '_weak4 Base.Option.t
= <fun>


In [12]:
PattnMatch.pair (PattnMatch.int_ (3)) PattnMatch.var

- : Core.Int.t * '_weak5 -> ('_weak5 -> '_weak6) -> '_weak6 Base.Option.t =
<fun>


In [13]:
PattnMatch.pair (PattnMatch.int_ (3)) PattnMatch.var (3, 2) (fun x -> x)

- : int Base.Option.t = Base.Option.Some 2


# Tagless Final, but with DB indices

Need to try to see if I can generate things with De Bruijn indices instead, and what it could look like. Current issue is that we're encoding the context $\Gamma$ as this curried function `'f`, and this is similar in spirit to doing HOAS. The issue, of course, is that I'd like a more explicit definition of variables so that I can manipulate them and rename them etc.

In [14]:
(x, y) \intersect (y, x) -> 
  (x, y)

  (x, y) -| x: int -> y: int -> 'r
  (y, x) -| x: int -> y: int -> 'r

  (2, 2)

error: compile_error

In [15]:
type _ polylist =
  | Nil: 'a polylist | Cons : 'a polylist * 'b -> 'b polylist

type _ polylist = Nil : 'a polylist | Cons : 'a polylist * 'b -> 'b polylist


# Heterogenous List

In [16]:
module Ctx_ = struct
  type _ var
  and _ code
  
  type _ context =
    | ( :: ) : 'hd var * 'tl context -> ('hd * 'tl) context
    | [] : unit context
  
  type ('g, 'd) renaming = 'g context -> 'd context
  let id : 'g. ('g, 'g) renaming = fun g -> g
  let compose : 'g 'd 'h. ('g, 'd) renaming -> ('d, 'h) renaming -> ('g, 'h) renaming =
    fun r1 r2 env -> r2 (r1 env)
  let weaken : 'a 'g. 'a var -> ('g, 'a * 'g) renaming = fun x g -> x :: g
  let swap : 'a 'b 'g. ('a * ('b * 'g), 'b * ('a * 'g)) renaming = fun (a :: b :: g) -> b :: a :: g
  
  type ('g, 'a) rhs = 'g context -> 'a code
  let rmap : 'a 'g 'd. ('g, 'd) renaming -> ('d, 'a) rhs -> ('g, 'a) rhs =
    fun r rhs env -> rhs (r env)  
end

module Ctx_ :
  sig
    type _ var
    and _ code
    type _ context =
        (::) : 'hd var * 'tl context -> ('hd * 'tl) context
      | [] : unit context
    type ('g, 'd) renaming = 'g context -> 'd context
    val id : ('g, 'g) renaming
    val compose : ('g, 'd) renaming -> ('d, 'h) renaming -> ('g, 'h) renaming
    val weaken : 'a var -> ('g, 'a * 'g) renaming
    val swap : ('a * ('b * 'g), 'b * ('a * 'g)) renaming
    type ('g, 'a) rhs = 'g context -> 'a code
    val rmap : ('g, 'd) renaming -> ('d, 'a) rhs -> ('g, 'a) rhs
  end


Unfortunately doesn't support appending lists 

## First Try: with context with a tail ptr

In [17]:
module Ctx = struct
  type 'a var = 'a code
  and 'a code = 'a

  type (_, _) context =
    | ( :: ) : 'hd var * ('tl, 'a) context -> ('hd * 'tl, 'a) context
    | [] : ('a, 'a) context

  type ('g, 'd, 'a) renaming = ('g, 'a) context -> ('d, 'a) context
  let id : 'g 'a. ('g, 'g, 'a) renaming = fun g -> g
  let compose : 'g 'd 'h 'a. ('g, 'd, 'a) renaming -> ('d, 'h, 'a) renaming -> ('g, 'h, 'a) renaming =
    fun r1 r2 env -> r2 (r1 env)
  let weaken : 'a 'g 'b. 'a var -> ('g, 'a * 'g, 'b) renaming = fun x g -> x :: g
  (* source of warning here. *)
  let swap : 'a 'b 'g 'c. ('a * ('b * 'g), 'b * ('a * 'g), 'c) renaming = fun (a :: b :: g) -> b :: a :: g

  let rec concat : type a b c. (a, b) context -> (b, c) context -> (a, c) context = fun c1 c2 ->
    match c1 with
    | x :: xs -> x :: concat xs c2
    | [] -> c2

  type ('g, 'a) rhs = ('g, unit) context -> 'a code
  let rmap : 'a 'g 'd 'b. ('g, 'd, unit) renaming -> ('d, 'a) rhs -> ('g, 'a) rhs =
    fun r rhs env -> rhs (r env)

end


File "[17]", line 15, characters 74-106:
15 |   let swap : 'a 'b 'g 'c. ('a * ('b * 'g), 'b * ('a * 'g), 'c) renaming = fun (a :: b :: g) -> b :: a :: g
                                                                               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
Warning 8 [partial-match]: this pattern-matching is not exhaustive.
Here is an example of a case that is not matched:
_::[]


module Ctx :
  sig
    type 'a var = 'a code
    and 'a code = 'a
    type (_, _) context =
        (::) : 'hd var * ('tl, 'a) context -> ('hd * 'tl, 'a) context
      | [] : ('a, 'a) context
    type ('g, 'd, 'a) renaming = ('g, 'a) context -> ('d, 'a) context
    val id : ('g, 'g, 'a) renaming
    val compose :
      ('g, 'd, 'a) renaming -> ('d, 'h, 'a) renaming -> ('g, 'h, 'a) renaming
    val weaken : 'a var -> ('g, 'a * 'g, 'b) renaming
    val swap : ('a * ('b * 'g), 'b * ('a * 'g), 'c) renaming
    val concat : ('a, 'b) context -> ('b, 'c) context -> ('a, 'c) context
    type ('g, 'a) rhs = ('g, unit) context -> 'a code
    val rmap : ('g, 'd, unit) renaming -> ('d, 'a) rhs -> ('g, 'a) rhs
  end


In [18]:
Ctx.swap Ctx.[]

error: runtime_error

In [19]:
Ctx.concat Ctx.(1 :: "s" :: []) Ctx.((fun x -> x) :: "e" :: [])

- : (int * (string * (('_weak7 -> '_weak7) * (string * '_weak8))), '_weak8)
    Ctx.context
= Ctx.(::) (1, Ctx.(::) ("s", Ctx.(::) (<fun>, Ctx.(::) ("e", Ctx.[]))))


This all works except type-checking doesn't for swap. Now I'll try to add type level numbers. https://www.kb.ecei.tohoku.ac.jp/ml2008/slides/lindley.pdf

In [20]:
module Pat = struct
  module Ctx = struct
    type 'a var = 'a code
    and 'a code = 'a
  
    type _ context =
      | ( :: ) : 'hd var * 'tl context -> ('hd * 'tl) context
      | [] : unit context
  
    type ('g, 'd) renaming = 'g context -> 'd context

    let rec concat : 'a context -> 'b context -> 
    let id : 'g. ('g, 'g) renaming = fun g -> g
    let compose : 'g 'd 'h. ('g, 'd) renaming -> ('d, 'h) renaming -> ('g, 'h) renaming =
      fun r1 r2 env -> r2 (r1 env)
    let weaken : 'a 'g. 'a var -> ('g, 'a * 'g) renaming = fun x g -> x :: g
    let swap : 'a 'b 'g. ('a * ('b * 'g), 'b * ('a * 'g)) renaming = fun (a :: b :: g) -> b :: a :: g
  
    type ('g, 'a) rhs = 'g context -> 'a code
    let rmap : 'a 'g 'd. ('g, 'd) renaming -> ('d, 'a) rhs -> ('g, 'a) rhs =
      fun r rhs env -> rhs (r env)
  end

  (* second argument changed to just the type of the context 'g *)
  type (_, _, _) pat = 
    Any : ('a, unit, 'r) pat
  | Int : int -> (int, unit, 'r) pat
  | Var : ('a, ('a * unit), 'r) pat (* Would be 'a code here instead *)
  | EmptyList : ('a list, unit, 'r) pat
  | Pair : ('a, 'k, 'j) pat * ('b, 'j, 'r) pat -> ('a * 'b, 'k, 'r) pat (* Breaks: Need way of concatenating two contexts. *)
  | Cons : ('a, 'k, 'j) pat * ('a list, 'j, 'r) pat -> ('a list, 'k, 'r) pat
  | Or : ('a, 'k, 'r) pat * ('a, 'k, 'r) pat -> ('a, 'k, 'r) pat
end

error: compile_error

## Adding context length

### Lindley's paper

https://www.kb.ecei.tohoku.ac.jp/ml2008/slides/lindley.pdf

In [21]:
type z
type +'n s

module Nat : sig
  type +'i t
  val zero : ('m * 'm) t
  val succ : ('m * 'n) t -> ('m * 'n s) t
  val add : ('m * 'n) t * ('l * 'm) t -> ('l * 'n) t 
  val to_int : 'i t -> int
end = struct
  type 'i t = int
  let zero = 0
  let succ n = n + 1
  let add (n,m) = n + m 
  let to_int n = n
end

type z


type +'n s


module Nat :
  sig
    type +'i t
    val zero : ('m * 'm) t
    val succ : ('m * 'n) t -> ('m * 'n s) t
    val add : ('m * 'n) t * ('l * 'm) t -> ('l * 'n) t
    val to_int : 'i t -> int
  end


In [22]:
Nat.(succ zero)

- : ('a * 'a s) Nat.t = <abstr>


In [23]:
Nat.(add (succ (succ zero), succ (succ zero)))

- : ('a * 'a s s s s) Nat.t = <abstr>


In [24]:
let f : 'a * 'a s s s s -> 'a = fun (a, b) -> a

val f : 'a * 'a s s s s -> 'a = <fun>


"syntactic non-values cannot be polymorphic". -- Ask Jeremy what this means.

The covariance annotations enable generalisation.

In [25]:
type z
type +'n s

module NList : sig
  type (+'length, +'elem_type) t
  val nil : ('m*'m, 'a) t
  val cons : 'a * ('m*'n, 'a) t -> ('m*'n s, 'a) t
  val append : ('m*'n, 'a) t * ('l*'m, 'a) t -> ('l*'n, 'a) t
  val to_list : ('i, 'a) t -> 'a list 
  val swap : ('n s s * 'm, 'a) t -> ('n s s * 'm, 'a) t
end = struct
  open List
  type ('i, 'a) t = 'a list
  let nil = []
  let cons (x, xs) = (x :: xs)
  let append (xs, ys) = xs @ ys
  let to_list xs = xs
  let swap (x::y::xs) = y::x::xs
end

File "[25]", line 15, characters 11-32:
15 |   let swap (x::y::xs) = y::x::xs
                ^^^^^^^^^^^^^^^^^^^^^
Warning 8 [partial-match]: this pattern-matching is not exhaustive.
Here is an example of a case that is not matched:
_::[]


module NList :
  sig
    type (+'length, +'elem_type) t
    val nil : ('m * 'm, 'a) t
    val cons : 'a * ('m * 'n, 'a) t -> ('m * 'n s, 'a) t
    val append : ('m * 'n, 'a) t * ('l * 'm, 'a) t -> ('l * 'n, 'a) t
    val to_list : ('i, 'a) t -> 'a list
    val swap : ('n s s * 'm, 'a) t -> ('n s s * 'm, 'a) t
  end


### Initial thing

In [26]:
module Ctx = struct
  type 'a var = 'a code
  and 'a code = 'a

  type z = unit
  type 'n s = 'n * unit

  type (_, _, _) context =
    | ( :: ) : 'hd var * ('n, 'tl, 'a) context -> ('n s, 'hd * 'tl, 'a) context
    | [] : (z, 'a, 'a) context

  (* n -> m renaming *)
  type ('n, 'm, 'g, 'd, 'a) renaming = ('n, 'g, 'a) context -> ('m, 'd, 'a) context
  let id : 'n 'g 'a. ('n, 'n, 'g, 'g, 'a) renaming = fun g -> g
  let compose : 'n 'm 'k 'g 'd 'h 'a. ('n, 'm, 'g, 'd, 'a) renaming -> ('m, 'k, 'd, 'h, 'a) renaming -> ('n, 'k, 'g, 'h, 'a) renaming =
    fun r1 r2 env -> r2 (r1 env)
  let weaken : 'n 'a 'g 'b. 'a var -> ('n, 'n s, 'g, 'a * 'g, 'b) renaming = fun x g -> x :: g
  (* source of warning here. *)
  let swap : 'n 'a 'b 'g 'c. ('n s s, 'n s s, 'a * ('b * 'g), 'b * ('a * 'g), 'c) renaming = fun (a :: b :: g) -> b :: a :: g

  (* let rec concat : type n m a b c. (n, a, b) context -> (b, c) context -> (a, c) context = fun c1 c2 ->
    match c1 with
    | x :: xs -> x :: concat xs c2
    | [] -> c2 *)

  type ('n, 'g, 'a) rhs = ('n, 'g, unit) context -> 'a code
  let rmap : 'a 'g 'd 'b 'n 'm. ('m, 'n, 'g, 'd, unit) renaming -> ('n, 'd, 'a) rhs -> ('m, 'g, 'a) rhs =
    fun r rhs env -> rhs (r env)
end

module Ctx :
  sig
    type 'a var = 'a code
    and 'a code = 'a
    type z = unit
    type 'n s = 'n * unit
    type (_, _, _) context =
        (::) : 'hd var *
          ('n, 'tl, 'a) context -> ('n s, 'hd * 'tl, 'a) context
      | [] : (z, 'a, 'a) context
    type ('n, 'm, 'g, 'd, 'a) renaming =
        ('n, 'g, 'a) context -> ('m, 'd, 'a) context
    val id : ('n, 'n, 'g, 'g, 'a) renaming
    val compose :
      ('n, 'm, 'g, 'd, 'a) renaming ->
      ('m, 'k, 'd, 'h, 'a) renaming -> ('n, 'k, 'g, 'h, 'a) renaming
    val weaken : 'a var -> ('n, 'n s, 'g, 'a * 'g, 'b) renaming
    val swap : ('n s s, 'n s s, 'a * ('b * 'g), 'b * ('a * 'g), 'c) renaming
    type ('n, 'g, 'a) rhs = ('n, 'g, unit) context -> 'a code
    val rmap :
      ('m, 'n, 'g, 'd, unit) renaming -> ('n, 'd, 'a) rhs -> ('m, 'g, 'a) rhs
  end


Now let's enable concatenation

In [27]:
module Ctx = struct
  type 'a var = 'a code
  and 'a code = 'a

  type z = unit
  type 'n s = 'n * unit

  type (_, _, _) context =
    | ( :: ) : 'hd var * ('n * 'm, 'tl, 'a) context -> ('n s * 'm, 'hd * 'tl, 'a) context
    | [] : ('m * 'm, 'a, 'a) context (* Another tail pointer here ;-; *)

  (* n -> m renaming *)
  type ('n, 'm, 'k, 'g, 'd, 'a) renaming = ('n * 'k, 'g, 'a) context -> ('m * 'k, 'd, 'a) context
  let id : 'n 'k 'g 'a. ('n, 'n, 'k, 'g, 'g, 'a) renaming = fun g -> g
  let compose : 'n 'm 'k 'g 'd 'h 'a 'l. ('n, 'm, 'l, 'g, 'd, 'a) renaming -> ('m, 'k, 'l, 'd, 'h, 'a) renaming -> ('n, 'k, 'l, 'g, 'h, 'a) renaming =
    fun r1 r2 env -> r2 (r1 env)
  let weaken : 'n 'k 'a 'g 'b. 'a var -> ('n, 'n s, 'k, 'g, 'a * 'g, 'b) renaming = fun x g -> x :: g
  (* source of warning here. *)
  let swap : 'n 'k 'a 'b 'g 'c. ('n s s, 'n s s,'k, 'a * ('b * 'g), 'b * ('a * 'g), 'c) renaming = fun (a :: b :: g) -> b :: a :: g

  (* let rec concat : type n m a b c. (n, a, b) context -> (b, c) context -> (a, c) context = fun c1 c2 ->
    match c1 with
    | x :: xs -> x :: concat xs c2
    | [] -> c2 *)

  (*type ('n, 'g, 'a) rhs = ('n, 'g, unit) context -> 'a code
  let rmap : 'a 'g 'd 'b 'n 'm. ('m, 'n, 'g, 'd, unit) renaming -> ('n, 'd, 'a) rhs -> ('m, 'g, 'a) rhs =
    fun r rhs env -> rhs (r env) *)
end

File "[27]", line 19, characters 99-131:
19 |   let swap : 'n 'k 'a 'b 'g 'c. ('n s s, 'n s s,'k, 'a * ('b * 'g), 'b * ('a * 'g), 'c) renaming = fun (a :: b :: g) -> b :: a :: g
                                                                                                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
Warning 8 [partial-match]: this pattern-matching is not exhaustive.
Here is an example of a case that is not matched:
_::[]


module Ctx :
  sig
    type 'a var = 'a code
    and 'a code = 'a
    type z = unit
    type 'n s = 'n * unit
    type (_, _, _) context =
        (::) : 'hd var *
          ('n * 'm, 'tl, 'a) context -> ('n s * 'm, 'hd * 'tl, 'a) context
      | [] : ('m * 'm, 'a, 'a) context
    type ('n, 'm, 'k, 'g, 'd, 'a) renaming =
        ('n * 'k, 'g, 'a) context -> ('m * 'k, 'd, 'a) context
    val id : ('n, 'n, 'k, 'g, 'g, 'a) renaming
    val compose :
      ('n, 'm, 'l, 'g, 'd, 'a) renaming ->
      ('m, 'k, 'l, 'd, 'h, 'a) renaming -> ('n, 'k, 'l, 'g, 'h, 'a) renaming
    val weaken : 'a var -> ('n, 'n s, 'k, 'g, 'a * 'g, 'b) renaming
    val swap :
      ('n s s, 'n s s, 'k, 'a * ('b * 'g), 'b * ('a * 'g), 'c) renaming
  end


**We're back to square 1 LOL**

In short, the problem is clear. 
- If we have a tail pointer, then we can't do swap properly, i.e. type checks but gives a partial match warning because the *base case* becomes of arbitrary type, and so saying `'n s s` doesn't actually help you make sure that you do have 2 elements on the front.
- If we don't have a tail pointer then we can't concatenate properly on the type level.

## Try again with original representation

In [39]:
module type Pat = sig
  type ('a, 'f, 'r) pat
  val int_ : int -> (int, 'r, 'r) pat
  val any_ : ('a, 'r, 'r) pat
  val var : ('a, 'a -> 'r, 'r) pat
  val emptylist : ('a list, 'r, 'r) pat
  val pair : ('a, 'k, 'j) pat -> ('b, 'j, 'r) pat -> ('a * 'b, 'k, 'r) pat
  val cons : ('a, 'k, 'j) pat -> ('a list, 'j, 'r) pat -> ('a list, 'k, 'r) pat
  val or_ : ('a, 'k, 'r) pat -> ('a, 'k, 'r) pat -> ('a, 'k, 'r) pat
  val rename : ('a, 'f -> 'g -> 'h, 'r) pat -> ('a, 'g -> 'f -> 'h, 'r) pat
end

module type Pat =
  sig
    type ('a, 'f, 'r) pat
    val int_ : int -> (int, 'r, 'r) pat
    val any_ : ('a, 'r, 'r) pat
    val var : ('a, 'a -> 'r, 'r) pat
    val emptylist : ('a list, 'r, 'r) pat
    val pair : ('a, 'k, 'j) pat -> ('b, 'j, 'r) pat -> ('a * 'b, 'k, 'r) pat
    val cons :
      ('a, 'k, 'j) pat -> ('a list, 'j, 'r) pat -> ('a list, 'k, 'r) pat
    val or_ : ('a, 'k, 'r) pat -> ('a, 'k, 'r) pat -> ('a, 'k, 'r) pat
    val rename : ('a, 'f -> 'g -> 'h, 'r) pat -> ('a, 'g -> 'f -> 'h, 'r) pat
  end


In [41]:
module PattnMatch : Pat = struct
  open Core.Option.Monad_infix
  type ('a, 'f, 'r) pat = 'a -> 'f -> 'r option
  let int_ i scrutinee cont = if i = scrutinee then Some (cont) else None
  let any_ scrutinee cont = Some cont
  let var scrutinee cont = Some (cont scrutinee)
  let emptylist scrutinee cont = Some cont
  let pair constr_left constr_right (scrutinee_left, scrutinee_right) cont = 
    constr_left scrutinee_left cont >>= fun m1 -> 
    constr_right scrutinee_right m1 >>= fun m2 -> 
    Some m2
  let cons constr_left constr_right (scrutinee) cont = 
    match scrutinee with 
    | scrutinee_left :: scrutinee_right ->
      constr_left scrutinee_left cont >>= fun m1 -> 
      constr_right scrutinee_right m1 >>= fun m2 ->
      Some m2
    | _ -> None
  let or_ constr_left constr_right scrutinee cont = 
      match constr_left scrutinee cont with
      | Some res -> Some res
      | None -> constr_right scrutinee cont
  let rename  = fun constr scrutinee cont -> constr scrutinee (fun x y -> cont y x)
  (* TODO: implement this sort of renaming *)
end

module PattnMatch : Pat


## CPS

In [47]:
module NList : sig
  type (+'length, +'elem_type) t_
  type (+'length, +'elem_type, 'b) t = (('length, 'elem_type) t_ -> 'b) -> 'b
  val nil : ('m*'m, 'a, 'b) t
  val cons : 'a * ('m*'n, 'a, 'b) t -> ('m*'n s, 'a, 'b) t
  val append : ('m*'n, 'a, 'b) t * ('l*'m, 'a, 'b) t -> ('l*'n, 'a, 'b) t
  val to_list : ('i, 'a, 'b) t -> 'a list 
  (* val swap : ('n s s * 'm, 'a) t -> ('n s s * 'm, 'a) t *)
end = struct
  open List
  type ('i, 'a) t_ = 'a list
  type (+'length, +'elem_type, 'b) t = (('length, 'elem_type) t_ -> 'b) -> 'b

  let nil = []
  let cons (x, xs) = (x :: xs)
  let append (xs, ys) = xs @ ys
  let to_list xs = xs
  (* let swap (x::y::xs) = y::x::xs *)
end

error: compile_error

## Recursive Definition of the function 'f = 'a -> 'a2 -> ... 'an -> 'r

In [128]:
module Ctx_bis = struct 
  type 'a var
  (* Logic behind this: context is a *function type* represented as HOAS that I can do manipulations on *)
  type (_, _) context = | Z : 'r -> ('r, 'r) context | S : ('a var -> ('g, 'r) context) -> ('a var -> 'g, 'r) context
  type ('g, 'd, 'r) renaming = ('g, 'r) context -> ('d, 'r) context
  let id : 'g. ('g, 'g, 'r) renaming = fun g -> g
  let compose : 'g 'd 'h 'r. ('g, 'd, 'r) renaming -> ('d, 'h, 'r) renaming -> ('g, 'h, 'r) renaming =
    fun r1 r2 env -> r2 (r1 env)
  let weaken : 'a 'g 'r. ('g, 'a var ->'g, 'r) renaming = fun g -> S (fun _ -> g)
  let swap : 'a 'b 'g 'r. ('a var -> ('b var -> 'g), 'b var -> ('a var -> 'g), 'r) renaming = fun (S (f)) -> S (fun b -> S (fun a -> match f a with (S g) -> g b))

  (*type ('g, 'a) rhs = 'g context -> 'a code
  let rmap : 'a 'g 'd. ('g, 'd) renaming -> ('d, 'a) rhs -> ('g, 'a) rhs =
    fun r rhs env -> rhs (r env)   *)
end

File "[128]", line 10, characters 133-160:
10 |   let swap : 'a 'b 'g 'r. ('a var -> ('b var -> 'g), 'b var -> ('a var -> 'g), 'r) renaming = fun (S (f)) -> S (fun b -> S (fun a -> match f a with (S g) -> g b))
                                                                                                                                          ^^^^^^^^^^^^^^^^^^^^^^^^^^^
Warning 8 [partial-match]: this pattern-matching is not exhaustive.
Here is an example of a case that is not matched:
Z _
File "[128]", line 10, characters 94-162:
10 |   let swap : 'a 'b 'g 'r. ('a var -> ('b var -> 'g), 'b var -> ('a var -> 'g), 'r) renaming = fun (S (f)) -> S (fun b -> S (fun a -> match f a with (S g) -> g b))
                                                                                                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
Warning 8 [partial-match]: this pattern-matching is not exhaustive.
Here is an example of a case that is not matched:
Z _


module Ctx_bis :
  sig
    type 'a var
    type (_, _) context =
        Z : 'r -> ('r, 'r) context
      | S : ('a var -> ('g, 'r) context) -> ('a var -> 'g, 'r) context
    type ('g, 'd, 'r) renaming = ('g, 'r) context -> ('d, 'r) context
    val id : ('g, 'g, 'r) renaming
    val compose :
      ('g, 'd, 'r) renaming -> ('d, 'h, 'r) renaming -> ('g, 'h, 'r) renaming
    val weaken : ('g, 'a var -> 'g, 'r) renaming
    val swap : ('a var -> 'b var -> 'g, 'b var -> 'a var -> 'g, 'r) renaming
  end


**TODO: Figure out why this is**

Say we have `('f, 'r) context`.

SMH the tag here seems to be the problem? Because this doesn't type check for general uses, but **it should** for the specific case where `'f` is used as an argument and not the return value, which is how we use it anyway. **Maybe try tagless final**.

Before this we try the original renaming on the pattern directly.

## Renaming original

Try to do renaming directly on pattern. Doesn't work because weakening can't be done compositionally. 

In [88]:
module type Pat = sig
  type ('a, 'f, 'r) pat
  val int_ : int -> (int, 'r, 'r) pat
  val any_ : ('a, 'r, 'r) pat
  val var : ('a, 'a -> 'r, 'r) pat
  val emptylist : ('a list, 'r, 'r) pat
  val pair : ('a, 'k, 'j) pat -> ('b, 'j, 'r) pat -> ('a * 'b, 'k, 'r) pat
  val cons : ('a, 'k, 'j) pat -> ('a list, 'j, 'r) pat -> ('a list, 'k, 'r) pat
  val or_ : ('a, 'k, 'r) pat -> ('a, 'k, 'r) pat -> ('a, 'k, 'r) pat
  val exchange : ('a, 'f -> 'g -> 'h, 'r) pat -> ('a, 'g -> 'f -> 'h, 'r) pat
  (* val weaken : ('a, 'h, 'r) pat -> ('a, 'f -> 'h, 'r) pat *)
end

module PattnMatch : Pat = struct
  open Core.Option.Monad_infix
  type ('a, 'f, 'r) pat = 'a -> 'f -> 'r option
  let int_ i scrutinee cont = if i = scrutinee then Some (cont) else None
  let any_ scrutinee cont = Some cont
  let var scrutinee cont = Some (cont scrutinee)
  let emptylist scrutinee cont = Some cont
  let pair constr_left constr_right (scrutinee_left, scrutinee_right) cont = 
    constr_left scrutinee_left cont >>= fun m1 -> 
    constr_right scrutinee_right m1 >>= fun m2 -> 
    Some m2
  let cons constr_left constr_right (scrutinee) cont = 
    match scrutinee with 
    | scrutinee_left :: scrutinee_right ->
      constr_left scrutinee_left cont >>= fun m1 -> 
      constr_right scrutinee_right m1 >>= fun m2 ->
      Some m2
    | _ -> None
  let or_ constr_left constr_right scrutinee cont = 
      match constr_left scrutinee cont with
      | Some res -> Some res
      | None -> constr_right scrutinee cont
  (* Doesn't work*)
  let exchange  = fun constr scrutinee cont -> constr scrutinee (fun x y -> cont y x)
  (* let weaken constr scrutinee cont = constr scrutinee (cont)  *)
  (* TODO: implement this sort of renaming *)
end

module type Pat =
  sig
    type ('a, 'f, 'r) pat
    val int_ : int -> (int, 'r, 'r) pat
    val any_ : ('a, 'r, 'r) pat
    val var : ('a, 'a -> 'r, 'r) pat
    val emptylist : ('a list, 'r, 'r) pat
    val pair : ('a, 'k, 'j) pat -> ('b, 'j, 'r) pat -> ('a * 'b, 'k, 'r) pat
    val cons :
      ('a, 'k, 'j) pat -> ('a list, 'j, 'r) pat -> ('a list, 'k, 'r) pat
    val or_ : ('a, 'k, 'r) pat -> ('a, 'k, 'r) pat -> ('a, 'k, 'r) pat
    val exchange :
      ('a, 'f -> 'g -> 'h, 'r) pat -> ('a, 'g -> 'f -> 'h, 'r) pat
  end


module PattnMatch : Pat


## Try to do tagless.

In [104]:
module type Ctx_tagless = sig
  type ('g, 'r) repr
  val z : ('r, 'r) repr
  val s : ('a -> ('g, 'r) repr) -> ('a -> 'g, 'r) repr
  val weaken : ('g, 'r) repr -> ('a -> 'g, 'r) repr
  val exchange : ('a -> 'b -> 'g, 'r) repr -> ('b -> 'a -> 'g, 'r) repr
  val concat : ('g, 'h) repr -> ('h, 'r) repr -> ('g, 'r) repr
end

module type Ctx_tagless =
  sig
    type ('g, 'r) repr
    val z : ('r, 'r) repr
    val s : ('a -> ('g, 'r) repr) -> ('a -> 'g, 'r) repr
    val weaken : ('g, 'r) repr -> ('a -> 'g, 'r) repr
    val exchange : ('a -> 'b -> 'g, 'r) repr -> ('b -> 'a -> 'g, 'r) repr
    val concat : ('g, 'h) repr -> ('h, 'r) repr -> ('g, 'r) repr
  end


In [107]:
module Ctx_tagless_impl : sig
  type ('g, 'r) repr
  val z : 'r -> ('r, 'r) repr
  val s : ('a -> ('g, 'r) repr) -> ('a -> 'g, 'r) repr
  val weaken : ('g, 'r) repr -> ('a -> 'g, 'r) repr
  val exchange : ('a -> 'b -> 'g, 'r) repr -> ('b -> 'a -> 'g, 'r) repr
  val concat : ('g, 'h) repr -> ('h, 'r) repr -> ('g, 'r) repr
end= struct
  (* forall 'r. ((ctx -> 'r) -> 'r) -> 'r*)
  type ('g, 'r) repr = 'g
  let z r = r
  let s f = f
  let weaken f _ = f
  let exchange f = fun b a -> f a b
  let concat (f : ('g, 'h) repr) (g: ('h, 'r) repr) = g (*Wrong*)
end

error: compile_error

In [124]:
module type Ctx_tagless = sig
  type ('g, 'r) repr
  val z : ('r, 'r) repr
  val s : 'a -> (('g, 'r) repr) -> ('a -> 'g, 'r) repr
  val weaken : ('g, 'r) repr -> ('a -> 'g, 'r) repr
  (*val exchange : ('a -> 'b -> 'g, 'r) repr -> ('b -> 'a -> 'g, 'r) repr
  val concat : ('g, 'h) repr -> ('h, 'r) repr -> ('g, 'r) repr *)
end
module Ctx_tagless_impl : Ctx_tagless = struct
  (* forall 'r. ((ctx -> 'r) -> 'r) -> 'r*)
  type ('g, 'h) repr = ('g -> 'h) (* Problem here: doesn't enforce the tail ptr. *)
  let z r = r
  let s a gr ag = gr (ag a)
  let weaken gr ag = ?
  (* let weaken f _ = f
  let exchange f = fun b a -> f a b
  let concat (f : ('g, 'h) repr) (g: ('h, 'r) repr) = g Wrong *)
end

module type Ctx_tagless =
  sig
    type ('g, 'r) repr
    val z : ('r, 'r) repr
    val s : 'a -> ('g, 'r) repr -> ('a -> 'g, 'r) repr
  end


module Ctx_tagless_impl : Ctx_tagless
